## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.10,80,94,23.02,light rain
1,1,Port Elizabeth,ZA,-33.9180,25.5701,52.09,87,0,6.91,clear sky
2,2,Komsomolskiy,UZ,40.4272,71.7189,79.52,38,0,9.22,clear sky
3,3,Bathurst,AU,-33.4166,149.5806,56.82,68,87,1.99,overcast clouds
4,4,Lebu,CL,-37.6167,-73.6500,50.99,89,99,11.03,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >=min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Komsomolskiy,UZ,40.4272,71.7189,79.52,38,0,9.22,clear sky
10,10,Baruun-Urt,MN,46.6806,113.2792,75.79,44,98,13.09,overcast clouds
12,12,Puerto Ayora,EC,-0.7393,-90.3518,75.16,93,53,5.99,broken clouds
15,15,Namatanai,PG,-3.6667,152.4333,84.00,68,88,5.91,overcast clouds
16,16,Bambanglipuro,ID,-7.9500,110.2833,76.55,82,98,9.98,overcast clouds
25,25,Kahului,US,20.8947,-156.4700,84.22,66,40,19.57,scattered clouds
28,28,Sinnamary,GF,5.3833,-52.9500,77.95,86,33,8.81,scattered clouds
29,29,Faanui,PF,-16.4833,-151.7500,77.27,73,13,10.87,light rain
30,30,Bathsheba,BB,13.2167,-59.5167,83.01,89,20,20.71,few clouds
37,37,Querecotillo,PE,-4.8378,-80.6456,77.07,47,24,9.80,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Country                212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Komsomolskiy,UZ,79.52,clear sky,40.4272,71.7189,
10,Baruun-Urt,MN,75.79,overcast clouds,46.6806,113.2792,
12,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
15,Namatanai,PG,84.00,overcast clouds,-3.6667,152.4333,
16,Bambanglipuro,ID,76.55,overcast clouds,-7.9500,110.2833,
25,Kahului,US,84.22,scattered clouds,20.8947,-156.4700,
28,Sinnamary,GF,77.95,scattered clouds,5.3833,-52.9500,
29,Faanui,PF,77.27,light rain,-16.4833,-151.7500,
30,Bathsheba,BB,83.01,few clouds,13.2167,-59.5167,
37,Querecotillo,PE,77.07,few clouds,-4.8378,-80.6456,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Komsomolskiy,UZ,79.52,clear sky,40.4272,71.7189,Ikathouse
10,Baruun-Urt,MN,75.79,overcast clouds,46.6806,113.2792,Ахмадын амралт сувилал
12,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Namatanai,PG,84.00,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
16,Bambanglipuro,ID,76.55,overcast clouds,-7.9500,110.2833,Puri Brata
...,...,...,...,...,...,...,...
675,Syracuse,IT,78.39,clear sky,37.0881,15.2792,Hotel Posta
676,Potamia,GR,75.51,clear sky,40.7163,24.7286,Blue Sea Beach Hotel
677,Manggar,ID,81.07,moderate rain,-2.8833,108.2667,Guest Hotel
678,Daokou,CN,80.74,overcast clouds,35.5639,114.5058,Shangri-La Hotel


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))